In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers, Input
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/https-www-kaggle-com-c-dcic-ia-2021-proyecto-ad/sample.csv
/kaggle/input/https-www-kaggle-com-c-dcic-ia-2021-proyecto-ad/train.csv
/kaggle/input/https-www-kaggle-com-c-dcic-ia-2021-proyecto-ad/test.csv


In [2]:
# Cargar el dataset para entrenamiento.
df = pd.read_csv('../input/https-www-kaggle-com-c-dcic-ia-2021-proyecto-ad/train.csv')

print("Cantidad de ejemplos: {}\nCantidad de columnas: {}".format(df.shape[0], df.shape[1]))

df.head(10)

Cantidad de ejemplos: 2714
Cantidad de columnas: 35


,id,country,league,season,home_team,away_team,home_team_buildUpPlaySpeed,home_team_buildUpPlaySpeedClass,home_team_buildUpPlayDribbling,home_team_buildUpPlayPassing,...,away_team_chanceCreationPassing,away_team_chanceCreationCrossing,away_team_chanceCreationShootingClass,away_team_chanceCreationPositioningClass,away_team_defencePressure,away_team_defenceAggression,away_team_defenceTeamWidth,away_team_defenceTeamWidthClass,away_team_defenceDefenderLineClass,match_result
0,2910,Spain,Spain LIGA BBVA,2015/2016,Rayo Vallecano,Getafe CF,29,Slow,52.0,44,...,55,39,Normal,Organised,43,42,49,Normal,Cover,1
1,3076,Switzerland,Switzerland Super League,2015/2016,FC Thun,FC St. Gallen,58,Balanced,59.0,35,...,39,39,Normal,Organised,40,36,52,Normal,Cover,-1
2,1178,Germany,Germany 1. Bundesliga,2015/2016,1. FC Köln,FC Bayern Munich,59,Balanced,47.0,65,...,37,31,Little,Free Form,72,53,59,Normal,Cover,-1
3,3128,Switzerland,Switzerland Super League,2015/2016,FC Sion,FC Thun,61,Balanced,57.0,37,...,64,36,Normal,Organised,47,45,53,Normal,Cover,1
4,1309,Italy,Italy Serie A,2015/2016,Hellas Verona,Roma,57,Balanced,29.0,38,...,71,54,Normal,Free Form,36,57,48,Normal,Cover,0
5,1357,Italy,Italy Serie A,2015/2016,Empoli,Lazio,66,Balanced,43.0,36,...,68,80,Lots,Free Form,59,66,65,Normal,Cover,1
6,2697,Spain,Spain LIGA BBVA,2015/2016,Málaga CF,Real Betis Balompié,42,Balanced,51.0,36,...,59,43,Normal,Organised,58,49,58,Normal,Cover,-1
7,1167,Germany,Germany 1. Bundesliga,2015/2016,Hertha BSC Berlin,FC Schalke 04,47,Balanced,44.0,55,...,64,61,Normal,Organised,55,59,56,Normal,Cover,1
8,2211,Portugal,Portugal Liga ZON Sagres,2015/2016,Vitória Guimarães,CD Nacional,40,Balanced,52.0,39,...,53,44,Normal,Organised,42,42,57,Normal,Cover,-1
9,2545,Scotland,Scotland Premier League,2015/2016,Dundee FC,Motherwell,47,Balanced,47.0,48,...,44,45,Little,Organised,37,44,51,Normal,Cover,0


In [3]:
# Columnas que tienen valores nulos (NaN)
print("Columnas con valores nulos:")
for column in df.columns:
    if df[column].isna().sum() > 0:
        print("-> {} => {} nulos".format(column, df[column].isna().sum()))

print("-"*50)

# Información sobre columnas categóricas
print("Columnas categóricas:")
for column in df.columns:
    if df[column].dtype == 'object':
        values = df[column].unique()
        print("-> {} => {} valores distintos".format(column, len(df[column].unique())))

print("-"*50)

# Distribución de los ejemplos (filas del dataset) para cada clase
print("Distribución de target (cantidad de ejemplos):")
print(df['match_result'].value_counts())

Columnas con valores nulos:
-> home_team_buildUpPlayDribbling => 39 nulos
-> away_team_buildUpPlayDribbling => 39 nulos
--------------------------------------------------
Columnas categóricas:
-> country => 11 valores distintos
-> league => 11 valores distintos
-> season => 1 valores distintos
-> home_team => 184 valores distintos
-> away_team => 184 valores distintos
-> home_team_buildUpPlaySpeedClass => 3 valores distintos
-> home_team_buildUpPlayPositioningClass => 2 valores distintos
-> home_team_chanceCreationShootingClass => 3 valores distintos
-> home_team_chanceCreationPositioningClass => 2 valores distintos
-> home_team_defenceTeamWidthClass => 3 valores distintos
-> home_team_defenceDefenderLineClass => 2 valores distintos
-> away_team_buildUpPlaySpeedClass => 3 valores distintos
-> away_team_buildUpPlayPositioningClass => 2 valores distintos
-> away_team_chanceCreationShootingClass => 3 valores distintos
-> away_team_chanceCreationPositioningClass => 2 valores distintos
-> a

In [4]:
mean_home_dribbling = df['home_team_buildUpPlayDribbling'].mean()
mean_away_dribbling = df['away_team_buildUpPlayDribbling'].mean()

df_filled = df.copy()

df_filled['home_team_buildUpPlayDribbling'] = df['home_team_buildUpPlayDribbling'].fillna(mean_home_dribbling)
df_filled['away_team_buildUpPlayDribbling'] = df['away_team_buildUpPlayDribbling'].fillna(mean_away_dribbling)

print("Verificando valores nulos:")
print(df_filled.isna().sum())

Verificando valores nulos:
id                                          0
country                                     0
league                                      0
season                                      0
home_team                                   0
away_team                                   0
home_team_buildUpPlaySpeed                  0
home_team_buildUpPlaySpeedClass             0
home_team_buildUpPlayDribbling              0
home_team_buildUpPlayPassing                0
home_team_buildUpPlayPositioningClass       0
home_team_chanceCreationPassing             0
home_team_chanceCreationCrossing            0
home_team_chanceCreationShootingClass       0
home_team_chanceCreationPositioningClass    0
home_team_defencePressure                   0
home_team_defenceAggression                 0
home_team_defenceTeamWidth                  0
home_team_defenceTeamWidthClass             0
home_team_defenceDefenderLineClass          0
away_team_buildUpPlaySpeed                  0
away_te

In [5]:
cols_eliminar = ["id", "country", "league", "season", "home_team", "away_team"] # Ingrese los nombres de las columnas a eliminar (entre comillas)

df_removed = df_filled.copy()
df_removed = df_filled.drop(columns = cols_eliminar)

print("Cantidad de ejemplos: {}\nCantidad de columnas: {}".format(df_removed.shape[0], df_removed.shape[1]))

Cantidad de ejemplos: 2714
Cantidad de columnas: 29


In [6]:
speed_class_map = {
    'Slow': 0,
    'Balanced': 1,
    'Fast': 2
}

positioned_class_map = {
    'Organised': 0,
    'Free Form': 1
}

shooting_class_map = {
    'Litte': 0,
    'Normal': 1,
    'Lots': 2
}

width_class_map = {
    'Narrow': 0,
    'Normal': 1,
    'Wide': 2
}

defenderline_class_map = {
    'Cover': 0,
    'Offside Trap': 1
}

#home_team_defenceDefenderLineClass => 2 valores distintos

df_mapped = df_removed.copy()

df_mapped['home_team_buildUpPlaySpeedClass'] = df_mapped['home_team_buildUpPlaySpeedClass'].map(speed_class_map)
df_mapped['away_team_buildUpPlaySpeedClass'] = df_mapped['away_team_buildUpPlaySpeedClass'].map(speed_class_map)

df_mapped['home_team_buildUpPlayPositioningClass'] = df_mapped['home_team_buildUpPlayPositioningClass'].map(positioned_class_map)
df_mapped['away_team_buildUpPlayPositioningClass'] = df_mapped['away_team_buildUpPlayPositioningClass'].map(positioned_class_map)

df_mapped['home_team_chanceCreationShootingClass'] = df_mapped['home_team_chanceCreationShootingClass'].map(shooting_class_map)
df_mapped['away_team_chanceCreationShootingClass'] = df_mapped['away_team_chanceCreationShootingClass'].map(shooting_class_map)

df_mapped['home_team_chanceCreationPositioningClass'] = df_mapped['home_team_chanceCreationPositioningClass'].map(positioned_class_map)
df_mapped['away_team_chanceCreationPositioningClass'] = df_mapped['away_team_chanceCreationPositioningClass'].map(positioned_class_map)

df_mapped['home_team_defenceTeamWidthClass'] = df_mapped['home_team_defenceTeamWidthClass'].map(width_class_map)
df_mapped['away_team_defenceTeamWidthClass'] = df_mapped['away_team_defenceTeamWidthClass'].map(width_class_map)

df_mapped['home_team_defenceDefenderLineClass'] = df_mapped['home_team_defenceDefenderLineClass'].map(defenderline_class_map)
df_mapped['away_team_defenceDefenderLineClass'] = df_mapped['away_team_defenceDefenderLineClass'].map(defenderline_class_map)

df_mapped.head(10)

,home_team_buildUpPlaySpeed,home_team_buildUpPlaySpeedClass,home_team_buildUpPlayDribbling,home_team_buildUpPlayPassing,home_team_buildUpPlayPositioningClass,home_team_chanceCreationPassing,home_team_chanceCreationCrossing,home_team_chanceCreationShootingClass,home_team_chanceCreationPositioningClass,home_team_defencePressure,...,away_team_chanceCreationPassing,away_team_chanceCreationCrossing,away_team_chanceCreationShootingClass,away_team_chanceCreationPositioningClass,away_team_defencePressure,away_team_defenceAggression,away_team_defenceTeamWidth,away_team_defenceTeamWidthClass,away_team_defenceDefenderLineClass,match_result
0,29,0,52.0,44,0,44,44,1.0,0,54,...,55,39,1.0,0,43,42,49,1,0,1
1,58,1,59.0,35,0,64,36,1.0,0,47,...,39,39,1.0,0,40,36,52,1,0,-1
2,59,1,47.0,65,0,57,36,2.0,0,42,...,37,31,NaN,1,72,53,59,1,0,-1
3,61,1,57.0,37,0,62,50,1.0,0,45,...,64,36,1.0,0,47,45,53,1,0,1
4,57,1,29.0,38,0,39,54,1.0,0,47,...,71,54,1.0,1,36,57,48,1,0,0
5,66,1,43.0,36,0,59,64,1.0,0,46,...,68,80,2.0,1,59,66,65,1,0,1
6,42,1,51.0,36,0,41,40,1.0,0,47,...,59,43,1.0,0,58,49,58,1,0,-1
7,47,1,44.0,55,0,59,68,1.0,0,50,...,64,61,1.0,0,55,59,56,1,0,1
8,40,1,52.0,39,0,53,62,1.0,0,37,...,53,44,1.0,0,42,42,57,1,0,-1
9,47,1,47.0,48,0,32,61,1.0,0,50,...,44,45,NaN,0,37,44,51,1,0,0
